# Handling dependencies

What if my "unit" uses system facilities?

- os.remove / datetime.now / etc.
- database access
- network connectivity
- user input

## Option 1: Dependency injection

Make the dependency part of the unit interface

In [1]:
import os, time
from collections import namedtuple
AuditRecord = namedtuple('AuditRecord', 'uid timestamp action')


def make_audit_record(action):
    return AuditRecord(os.getuid(), time.time(), action)

In [2]:
make_audit_record('update')

AuditRecord(uid=501, timestamp=1584618908.2130508, action='update')

Depends on: current user, current time (hard to test)

### With Dependency injection

In [3]:
def make_audit_record(action, time=time.time, getuid=os.getuid):
    return AuditRecord(getuid(), time(), action)    

In [4]:
def mytime():
    return 200

def mygetuid():
    return 1024

# make_audit_record('update', time=lambda: 200, getuid=lambda: 1024)
make_audit_record('update', time=mytime, getuid=mygetuid)

AuditRecord(uid=1024, timestamp=200, action='update')

Test can inject new versions of utcnow and getuid and test to ensure things work as expected. Unfortunately, we've "uglified" our interface.

## Option 2: Mocking and patching

The `unittest.mock` library provides the ability to patch dependencies:

In [5]:
# original implementation

def make_audit_record(action):
    return AuditRecord(os.getuid(), time.time(), action)

In [6]:
# Test code
import os, time
from unittest.mock import patch

with patch('os.getuid') as p_getuid, patch('time.time') as p_time:
    p_getuid.return_value = 1024
    p_time.return_value = 200
    rec = make_audit_record('update')
rec

AuditRecord(uid=1024, timestamp=200, action='update')

In [7]:
os.getuid(), time.time()

(501, 1584619081.299469)

patch -> guerilla patch -> monkey patch

# Database Mocking

Depending on how much DBMS-specific SQL you use, you *may* be able to use Python's builtin `sqlite` database as a mock:

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

data = pd.read_csv('../data/closing-prices.csv', index_col=0, parse_dates=[0])
data.head()

## We can dump the dataframe to an in-memory (SQL) database

In [ ]:
import sqlite3

conn = sqlite3.connect(':memory:')
data.to_sql('prices', conn)

In [ ]:
for row in conn.execute('SELECT * FROM prices LIMIT 5'):
    print(row)

# We can read it back into a dataframe

In [ ]:
data2 = pd.read_sql('SELECT * FROM prices WHERE IBM > 160', conn)
data2.info()

In [ ]:
data2.head()

# More mock examples

In [8]:
from unittest import mock

m = mock.Mock()
print(m.whatever)

<Mock name='mock.whatever' id='4545609936'>


In [9]:
m.a.return_value = 42

In [10]:
m.a()

42

In [11]:
%%file data/test-examples/test8.py
import unittest
from unittest import mock


def echo_data(socket):
    data = socket.recv(42)
    socket.send(data)


class MyTest(unittest.TestCase):

    def test_send_recv(self):
        socket = mock.Mock()
        socket.recv.return_value = 'Some data'
        echo_data(socket)
        socket.send.assert_called_with('Some data')


Overwriting data/test-examples/test8.py


In [12]:
!python -m unittest data/test-examples/test8.py

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


In [13]:
from unittest import mock

In [14]:
m = mock.Mock(name='bertrand')

In [15]:
m

<Mock name='bertrand' id='4544956560'>

In [16]:
m.foo

<Mock name='bertrand.foo' id='4544874064'>

In [17]:
m()

<Mock name='bertrand()' id='4545648016'>

In [18]:
m.foo.return_value = 16
m.foo()

16

In [19]:
m.foo.assert_called_with()

In [ ]:
m.side_effect = ValueError

In [ ]:
m()

In [ ]:
m[10]

In [ ]:
m + m

In [ ]:
m = mock.MagicMock(name='russell')

In [ ]:
m.__add__.return_value = 'foo'

In [ ]:
m + m

In [ ]:
m + None

In [ ]:
m - 5

In [20]:
from datetime import datetime

In [ ]:
datetime.utcnow()

In [ ]:
datetime.gmtnow()

In [ ]:
mock_dt = mock.Mock()

In [ ]:
mock_dt.utcnow()

In [ ]:
mock_dt.gmtnow()

In [ ]:
mock_dt.gmtnow.side_effect = AttributeError

In [ ]:
mock_dt.gmtnow()

In [21]:
mock_dt = mock.create_autospec(datetime)

In [22]:
mock_dt.utcnow()

<MagicMock name='mock.utcnow()' id='4543067408'>

In [23]:
mock_dt.gmtnow()

AttributeError: Mock object has no attribute 'gmtnow'

In [ ]:
mock_dt.utcnow('this is a spurious argument')

In [ ]:
mock_dt.utcnow.return_value = datetime(2011, 1, 1)

In [ ]:
mock_dt.utcnow()

In [ ]:
mock_dt.utcnow.assert_called_with()

In [ ]:
# import boto3
# ec2 = boto3.resource('ec2')
ec2 = mock.Mock()
ec2.meta.client.describe_instances.return_value = {
    'Instances': [
        {'id': ...}
    ]
}

ec2.meta.client.describe_instances()